In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

2024-09-19 19:32:19.823436: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 19:32:19.911416: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 19:32:20.002302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 19:32:20.076350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 19:32:20.098793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 19:32:20.236484: I tensorflow/core/platform/cpu_feature_gu

In [2]:
import pandas as pd
dataset=pd.read_csv("stream_cipher_dataset.csv")

In [3]:
def hex_to_bytes(hex_string, max_len=256):
    byte_seq = bytes.fromhex(hex_string)
    byte_seq = byte_seq.ljust(max_len, b'\x00')[:max_len]
    return byte_seq

In [4]:
max_len = 256
byte_sequences = np.array([np.frombuffer(hex_to_bytes(seq, max_len), dtype=np.uint8).reshape(16, 16, 1) 
                           for seq in dataset['byte_sequence']])

In [5]:
label_encoder = LabelEncoder()
algorithms = label_encoder.fit_transform(dataset['algorithm'])

In [6]:
y = to_categorical(algorithms)
print(byte_sequences.shape) 
print(y.shape)

(10000, 16, 16, 1)
(10000, 3)


In [16]:

def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, kernel_size=(2, 2), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [18]:
input_shape = (16, 16, 1)
num_classes = y.shape[1]
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.summary()

/media/narayanan/New Volume/sih/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 16, 16, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,763 (460.01 KB)

 Trainable params: 117,763 (460.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
input_shape = (16, 16, 1)
num_classes = y.shape[1]
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.summary()
history=cnn_model.fit(byte_sequences, y, epochs=10, batch_size=512, validation_split=0.2)

In [ ]:
history.save('model.h5')

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM,GRU
from keras.layers import Dense

In [8]:
from tensorflow import keras
from keras.layers import *

In [9]:
model=Sequential()
model.add(LSTM(100,input_shape=(16,16),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(100,return_sequences=False))
model.add(Dense(32))
model.add(Dense(3,activation='sigmoid'))

/media/narayanan/New Volume/sih/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 16, 100)        │        46,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         3,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,931 (823.95 KB)

 Trainable params: 210,931 (823.95 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
byte_sequences.reshape(10000,16,16)

array([[[251, 194,  69, ...,  52,  58, 155],
        [175, 146,  45, ..., 136, 236, 143],
        [107, 201, 169, ...,  28, 175, 166],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[212, 249, 195, ..., 189,  80, 136],
        [103,   2, 102, ...,  20, 219, 116],
        [160,  48, 123, ...,  30, 187,  29],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[144, 129, 245, ...,   6, 166, 244],
        [170,   7, 214, ...,   3,  38, 175],
        [ 24,  21,  34, ...,  20, 120, 141],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       ...,

       [[  1,  90, 220, ...,  38,  93,  92],
        [ 71,  27, 167, ..., 243, 199,  48],
        [ 71, 151, 119, ...,  35, 141, 245

In [14]:
model.fit(byte_sequences,y,epochs=10,batch_size=32,validation_batch_size=0.2,)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.3343 - loss: 1.1048
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3309 - loss: 1.0994
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3287 - loss: 1.0993
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3222 - loss: 1.0994
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3455 - loss: 1.0982
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3364 - loss: 1.0992
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.3379 - loss: 1.0987
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.3494 - loss: 1.0981
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3350 - loss: 1.0988
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.3319 - loss: 1.0988


In [13]:
model=Sequential()
model.add(GRU(128,input_shape=(16,16),return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(256,return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(512,return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(1024,return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(1024,return_sequences=False))
model.add(Dense(512))
model.add(Dense(3,activation='tanh'))

In [14]:
model.compile(optimizer='adamax',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(byte_sequences,y,epochs=10,batch_size=512,validation_batch_size=0.2)

Epoch 1/10


2024-09-19 19:33:14.900015: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-09-19 19:33:14.912206: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-09-19 19:33:14.921847: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-09-19 19:33:14.946350: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-09-19 19:33:16.642823: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.


20/20 ━━━━━━━━━━━━━━━━━━━━ 113s 5s/step - accuracy: 0.3252 - loss: 1.9261
Epoch 2/10
